# Ambiente

In [ ]:
# !pip install -r requirements.txt

# import capymoa
# print(capymoa.__version__)

# Base de Dados

## CICDDoS2019

### Downsample [Personalizado] 

In [ ]:
import pandas as pd
import os
import numpy as np

DATASET_PATH = 'datasets/CICDDoS2019/'
PANDAS_CHUNK_SIZE = 100000 
ATTACK_LABEL_COL = ' Label' 
COLUMNS_TO_DROP = ['Unnamed: 0', 'Flow ID', 'SimillarHTTP']
MIN_SAMPLES_PER_CHUNK = 1000
BENIGN_LABEL = 'BENIGN'
DOWNSAMPLE_FACTORS = {
    'TFTP': 0.001,  # Redução mais agressiva 
    'MSSQL': 0.001,
    'Default': 0.01  
}

ATTACK_ORDER = {
    '03-11': [
        'Portmap.csv', 'NetBIOS.csv', 'LDAP.csv', 'MSSQL.csv', 'UDP.csv', 'UDPLag.csv', 'Syn.csv'
    ],
    '01-12': [
        'DrDoS_NTP.csv', 'DrDoS_DNS.csv', 'DrDoS_LDAP.csv', 'DrDoS_MSSQL.csv', 'DrDoS_NetBIOS.csv', 'DrDoS_SNMP.csv', 'DrDoS_SSDP.csv', 'DrDoS_UDP.csv', 
        'UDPLag.csv', 'Syn.csv', 'TFTP.csv' 
    ]
}

OUTPUT_FILES = {
    '03-11': 'CICDDoS2019_03_11_Test.csv',
    '01-12': 'CICDDoS2019_01_12_Train.csv'
}

In [ ]:
def processar_e_salvar_dia(dia, dataset_path, chunk_size, cols_to_drop, attack_order, output_files):
    
    lista_arquivos = attack_order[dia]
    output_filepath = output_files[dia]
    total_amostras_mantidas = 0
    header_escrito = False

    with open(output_filepath, 'w', newline='', encoding='utf-8') as f:
        
        for filename in lista_arquivos:
            filepath = os.path.join(dataset_path, dia, filename)
            attack_name_from_file = filename.replace('.csv', '')
            
            try:
                csv_reader = pd.read_csv(
                    filepath, 
                    chunksize=chunk_size, 
                    low_memory=False, 
                    on_bad_lines='skip'
                )
            except Exception:
                continue
                
            for df_chunk in csv_reader:
                
                # Limpeza de Colunas
                cols_existentes_drop = [col for col in cols_to_drop if col in df_chunk.columns]
                df_chunk = df_chunk.drop(columns=cols_existentes_drop, errors='ignore')
                
                # Normalização do Rótulo
                df_chunk[ATTACK_LABEL_COL] = df_chunk[ATTACK_LABEL_COL].apply(
                    lambda x: BENIGN_LABEL if 'BENIGN' in str(x).upper() else attack_name_from_file
                )
                
                df_benign = df_chunk[df_chunk[ATTACK_LABEL_COL] == BENIGN_LABEL]
                df_ataque = df_chunk[df_chunk[ATTACK_LABEL_COL] != BENIGN_LABEL]

                # 3. Downsampling Seletivo nos Ataques
                df_ataque_downsampled = df_ataque

                if not df_ataque.empty:
                    # Fator de Downsample específico
                    factor = DOWNSAMPLE_FACTORS.get(attack_name_from_file, DOWNSAMPLE_FACTORS['Default'])
                    
                    # Se o número de amostras for MENOR que o mínimo, não fazemos Downsample (mantemos 100%)
                    if len(df_ataque) < MIN_SAMPLES_PER_CHUNK:
                        factor = 1.0
                    
                    if factor < 1.0:
                        df_ataque_downsampled = df_ataque.sample(
                            frac=factor, 
                            random_state=42
                        )
                    # Senão, df_ataque_downsampled permanece df_ataque (100% mantido)

                # Combina (BENIGN 100% + Ataque Reduzido) e Embaralha
                df_chunk_reduzido = pd.concat([df_benign, df_ataque_downsampled]).sample(frac=1, random_state=42)
                
                if df_chunk_reduzido.empty:
                    continue

                # Salvamento do Chunk Imediato
                df_chunk_reduzido.to_csv(
                    f, 
                    index=False, 
                    header=not header_escrito, 
                    mode='a' 
                )
                header_escrito = True
                total_amostras_mantidas += len(df_chunk_reduzido)

    return total_amostras_mantidas

In [ ]:
# tamanho_teste = processar_e_salvar_dia(
#     dia='03-11', 
#     dataset_path=DATASET_PATH,
#     chunk_size=PANDAS_CHUNK_SIZE,
#     cols_to_drop=COLUMNS_TO_DROP,
#     attack_order=ATTACK_ORDER,
#     output_files=OUTPUT_FILES
# )
# print(f"Dataset de Teste salvo: {OUTPUT_FILES['03-11']} (Total: {tamanho_teste:,} amostras)")

# tamanho_treino = processar_e_salvar_dia(
#     dia='01-12', 
#     dataset_path=DATASET_PATH,
#     chunk_size=PANDAS_CHUNK_SIZE,
#     cols_to_drop=COLUMNS_TO_DROP,
#     attack_order=ATTACK_ORDER,
#     output_files=OUTPUT_FILES
# )
# print(f"Dataset de Treino salvo: {OUTPUT_FILES['01-12']} (Total: {tamanho_treino:,} amostras)")

Dataset de Teste salvo: CICDDoS2019_03_11_Test.csv (Total: 208,102 amostras)
Dataset de Treino salvo: CICDDoS2019_01_12_Train.csv (Total: 376,216 amostras)


### Pré-processamento

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from capymoa.stream import NumpyStream

FILE_TRAIN = r'datasets\CICDDoS2019\CICDDoS2019_01_12_Train.csv'
FILE_TEST = r'datasets\CICDDoS2019\CICDDoS2019_03_11_Test.csv'
TARGET_LABEL = ' Label'
TIMESTAMP_COLUMN = ' Timestamp'

#### Funções de pré processamento

In [ ]:
def preprocess_stream_dataframe(df, target_label_col, timestamp_col, le_fit=None, is_train=False):
    print("=" * 60)
    df_processed = df.copy()
    
    if timestamp_col in df_processed.columns:
        print(f"Convertendo coluna '{timestamp_col}' para datetime...")
        df_processed[timestamp_col] = pd.to_datetime(df_processed[timestamp_col], errors='coerce')
    else:
        print(f"Aviso: Coluna de timestamp '{timestamp_col}' não encontrada.")
    
    # Trata valores infinitos
    df_processed.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    # Remove colunas 'object' desnecessárias
    cols_to_drop_objects = [col for col in df_processed.columns 
                            if df_processed[col].dtype == 'object' and col != target_label_col]
    
    if cols_to_drop_objects:
        print(f"Removendo colunas 'object': {cols_to_drop_objects}")
        df_processed.drop(columns=cols_to_drop_objects, errors='ignore', inplace=True)

    le = le_fit
    
    if target_label_col in df_processed.columns:
        if is_train:
            print("Codificando rótulos (modo treino)...")
            le = LabelEncoder()
            df_processed[target_label_col] = le.fit_transform(df_processed[target_label_col].astype(str))
        elif le_fit is not None:
            print("Codificando rótulos (modo teste/predição)...")
            
            def transform_label(label):
                try:
                    # Tenta transformar rótulo conhecido
                    return le.transform([label])[0]
                except ValueError:
                    # Atribui um novo índice para rótulo desconhecido
                    return len(le.classes_) 
            
            df_processed[target_label_col] = df_processed[target_label_col].astype(str).apply(transform_label)
        else:
            print(f"Aviso: 'is_train=False' mas nenhum 'le_fit' foi fornecido. Rótulos não serão codificados.")
            
    # Preenchimento de Nulos 
    print("Preenchendo valores ausentes (NaN/NaT) com 0...")
    df_processed.fillna(0, inplace=True)
    
    # Ordenação 
    if timestamp_col in df_processed.columns:
        print(f"Ordenando DataFrame por '{timestamp_col}'...")
        df_processed.sort_values(by=timestamp_col, inplace=True)
        df_processed.reset_index(drop=True, inplace=True)

    # Mover Rótulo para o Final
    if target_label_col in df_processed.columns:
        print(f"Movendo coluna alvo '{target_label_col}' para o final...")
        label_col_data = df_processed.pop(target_label_col)
        df_processed[target_label_col] = label_col_data
        
    print(f"Pré-processamento concluído. Shape final: {df_processed.shape}")
    print("=" * 60)
    
    return df_processed, le


def analisar_qualidade_dados(df, nome_df):
    print("=" * 60)
    print("\nValores Nulos (NaN) por Coluna:")
    nan_counts = df.isnull().sum()
    nan_counts_filtered = nan_counts[nan_counts > 0].sort_values(ascending=False)
    
    if nan_counts_filtered.empty:
        print("Nenhum valor nulo (NaN) encontrado.")
    else:
        print(nan_counts_filtered)

    # --- Contagem de Valores Infinitos (inf ou -inf) ---
    df_numeric = df.select_dtypes(include=np.number)
    
    print("\nValores Infinitos (inf/-inf) por Coluna:")
    if df_numeric.empty:
        print("Nenhuma coluna numérica encontrada para checar infinitos.")
    else:
        # np.isinf() aplicado ao DataFrame numérico e somado por coluna
        is_inf = np.isinf(df_numeric).sum()
        is_inf_filtered = is_inf[is_inf > 0].sort_values(ascending=False)
        
        if is_inf_filtered.empty:
            print("Nenhum valor infinito (inf/-inf) encontrado.")
        else:
            print(is_inf_filtered)

    print("=" * 60)

def criar_stream(df, target_label_col, timestamp_col, stream_name=""):
    print(f"Iniciando criação do stream de dados: {stream_name}")

    cols_to_drop_from_X = [target_label_col, timestamp_col]
    y_data = df[target_label_col].values
    X_data = df.drop(columns=cols_to_drop_from_X).values 
    feature_names = list(df.drop(columns=cols_to_drop_from_X).columns)

    # Criar o Stream
    stream = NumpyStream(
        X=X_data,
        y=y_data,
        feature_names=feature_names,
        target_name=target_label_col
    )
    stream.restart()

    return stream

#### Análise dos dados

In [2]:
# Treino
print("=== Treino ===")
df_train = pd.read_csv(FILE_TRAIN)
print(df_train.value_counts(' Label'))

# Teste
print("\n=== Teste ===")
df_test = pd.read_csv(FILE_TEST)
print(df_test.value_counts(' Label'))

=== Treino ===
 Label
BENIGN           56863
DrDoS_SNMP       51606
DrDoS_DNS        50715
DrDoS_MSSQL      45229
DrDoS_NetBIOS    40936
DrDoS_UDP        31349
DrDoS_SSDP       26109
DrDoS_LDAP       21800
TFTP             20087
Syn              15824
DrDoS_NTP        12029
UDPLag            3669
Name: count, dtype: int64

=== Teste ===
 Label
BENIGN     56965
Syn        42852
UDP        37795
NetBIOS    34549
LDAP       21083
UDPLag      7212
MSSQL       5776
Portmap     1870
Name: count, dtype: int64


#### Verificação de valores Inf e Ausentes / Ordenação Timestamp

In [4]:
analisar_qualidade_dados(df_train, "df_train (ANTES da Limpeza)")

# Limpa os dados de treino e "treina" o codificador de rótulos
df_train_cleaned, le_trained = preprocess_stream_dataframe(
    df=df_train,
    target_label_col=TARGET_LABEL,
    timestamp_col=TIMESTAMP_COLUMN,
    is_train=True
)

analisar_qualidade_dados(df_train_cleaned, "df_train_cleaned (APÓS a Limpeza)")
is_sorted_train = df_train_cleaned[TIMESTAMP_COLUMN].is_monotonic_increasing
print(f"Train ordenado por Timestamp: {is_sorted_train}")


Valores Nulos (NaN) por Coluna:
Flow Bytes/s    2462
dtype: int64

Valores Infinitos (inf/-inf) por Coluna:
 Flow Packets/s    8882
Flow Bytes/s       6420
dtype: int64
Convertendo coluna ' Timestamp' para datetime...
Removendo colunas 'object': [' Source IP', ' Destination IP']
Codificando rótulos (modo treino)...
Preenchendo valores ausentes (NaN/NaT) com 0...
Ordenando DataFrame por ' Timestamp'...
Movendo coluna alvo ' Label' para o final...
Pré-processamento concluído. Shape final: (376216, 83)

Valores Nulos (NaN) por Coluna:
Nenhum valor nulo (NaN) encontrado.

Valores Infinitos (inf/-inf) por Coluna:
Nenhum valor infinito (inf/-inf) encontrado.
Train ordenado por Timestamp: True


In [5]:
analisar_qualidade_dados(df_test, "df_test (ANTES da Limpeza)")

df_test_cleaned, _ = preprocess_stream_dataframe(
    df=df_test,
    target_label_col=TARGET_LABEL,
    timestamp_col=TIMESTAMP_COLUMN,
    le_fit=le_trained,
    is_train=False
)

analisar_qualidade_dados(df_test_cleaned, "df_test_cleaned (APÓS a Limpeza)")
is_sorted_test = df_test_cleaned[TIMESTAMP_COLUMN].is_monotonic_increasing
print(f"Test ordenado por Timestamp: {is_sorted_test}")


Valores Nulos (NaN) por Coluna:
Flow Bytes/s    46
dtype: int64

Valores Infinitos (inf/-inf) por Coluna:
 Flow Packets/s    7005
Flow Bytes/s       6959
dtype: int64
Convertendo coluna ' Timestamp' para datetime...
Removendo colunas 'object': [' Source IP', ' Destination IP']
Codificando rótulos (modo teste/predição)...
Preenchendo valores ausentes (NaN/NaT) com 0...
Ordenando DataFrame por ' Timestamp'...
Movendo coluna alvo ' Label' para o final...
Pré-processamento concluído. Shape final: (208102, 83)

Valores Nulos (NaN) por Coluna:
Nenhum valor nulo (NaN) encontrado.

Valores Infinitos (inf/-inf) por Coluna:
Nenhum valor infinito (inf/-inf) encontrado.
Test ordenado por Timestamp: True


## CICIDS2017

In [ ]:
import pandas as pd

df = pd.read_csv('datasets\CIC-IDS-2017\Wednesday-workingHours.pcap_ISCX.csv')
df.head()

 Label
BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,80,38308,1,1,6,6,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,389,479,11,5,172,326,79,0,15.636364,31.449238,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,88,1095,10,6,3150,3150,1575,0,315.000000,632.561635,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,389,15206,17,12,3452,6660,1313,0,203.058823,425.778474,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,88,1092,9,6,3150,3152,1575,0,350.000000,694.509719,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [11]:
print(df.value_counts(' Label'))

 Label
BENIGN              440031
DoS Hulk            231073
DoS GoldenEye        10293
DoS slowloris         5796
DoS Slowhttptest      5499
Heartbleed              11
Name: count, dtype: int64


# Algoritmos

## LeveragingBagging

https://capymoa.org/api/modules/capymoa.classifier.LeveragingBagging.html

In [33]:
from capymoa.classifier import LeveragingBagging
from capymoa.evaluation import prequential_evaluation
from capymoa.stream import stream_from_file
from capymoa.evaluation import ClassificationEvaluator
import math
import time
import warnings
warnings.filterwarnings('ignore') 

### Criação dos dados stream

In [17]:
# Chamar a função para criar o stream 
train_stream = criar_stream(
    df=df_train_cleaned,
    target_label_col=TARGET_LABEL,
    timestamp_col=TIMESTAMP_COLUMN,
    stream_name="Treino"
)

test_stream = criar_stream(
    df=df_test_cleaned,
    target_label_col=TARGET_LABEL,
    timestamp_col=TIMESTAMP_COLUMN,
    stream_name="Teste"
)


Iniciando criação do stream de dados: Treino

Iniciando criação do stream de dados: Teste


### Treinamento e Testando o modelo

In [38]:
# Inicializa o classificador com o esquema do stream
classifier = LeveragingBagging(schema=train_stream.get_schema())

# AVALIAÇÃO NO CONJUNTO DE TREINO
print("Iniciando Treinamento...")
start_train_time = time.time()

results_train = prequential_evaluation(
    stream=train_stream,
    learner=classifier,
    batch_size=1
)

end_train_time = time.time() 
training_duration = end_train_time - start_train_time

print("\nResultados de Avaliação (Treino):")
print(f"Acurácia Cumulativa: {results_train['cumulative'].accuracy():.4f}")
print(f"F1-Score Cumulativo: {results_train['cumulative'].f1_score():.4f}")
print(f"Tempo de Treinamento: {training_duration:.2f} segundos")


Iniciando Treinamento...

Resultados de Avaliação (Treino):
Acurácia Cumulativa: 99.0519
F1-Score Cumulativo: 98.3268
Tempo de Treinamento: 314.32 segundos


In [30]:
# AVALIAÇÃO NO CONJUNTO DE TESTE
print("\nIniciando Teste...")
test_stream.restart() # Resetar o stream de teste para o início 

# Criar um novo objeto avaliador para o teste
test_evaluator = ClassificationEvaluator(schema=test_stream.get_schema())

# Loop manual de TESTE 
start_test_time = time.time()
while test_stream.has_more_instances():
    instance = test_stream.next_instance()
    prediction = classifier.predict(instance)
    test_evaluator.update(instance.y_index, prediction)

end_test_time = time.time() 
test_duration = end_test_time - start_test_time

print("\nResultados de Avaliação (Teste):")
print(f"Acurácia Cumulativa: {test_evaluator.accuracy():.4f}")
print(f"F1-Score Cumulativo: {test_evaluator.f1_score():.4f}")
print(f"Tempo de Teste: {test_duration:.2f} segundos")


Iniciando Teste...

Resultados de Avaliação (Teste):
Acurácia Cumulativa: 27.3736
F1-Score Cumulativo: nan
Tempo de Teste: 57.44 segundos


## HoeffdingAdaptiveTree

https://capymoa.org/api/modules/capymoa.classifier.HoeffdingAdaptiveTree.html